In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM, Dense, Dropout # For Dense import, all added layers are neurons in one layer that're connected as inputs to all the inputs in the next layer.
# For Dense import, weights remove the neurons that don't need to be connected. These values get pushed down close to 0.
from tensorflow.keras.optimizers import Adam, SGD
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras.layers import InputLayer
from sklearn.preprocessing import Normalizer
%matplotlib inline

In [7]:
import yfinance as yf
amd = yf.Ticker("AMD")
amd_hist = amd.history(period="13y")
#amd_hist
#amd.info # Stored in a dictionary with all kinds of info on the company

In [9]:
amd_df = amd_hist.drop(columns=['Stock Splits', 'Dividends', 'Open', 'Volume'])
#amd_df

In [10]:
# Split for features & target
X = amd_df.iloc[:, 0:3]
y = amd_df.iloc[:,-1]
# X = pd.get_dummies(X) Not needed since all columns are already numerical values
# Training gets 80%, testing gets 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)
# Normalize works better 
y_column_transform = ColumnTransformer([('normalize', Normalizer(), [('Close')])])
X_column_transform = ColumnTransformer([('normalize', Normalizer(), ['High','Low','Close'])], remainder='passthrough')

In [11]:
X_train_scaled = X_column_transform.fit_transform(X_train)
X_test_scaled = X_column_transform.transform(X_test)

In [12]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)

In [13]:
#print(X_test_scaled.describe())
X_test_scaled.shape

(655, 3)

In [14]:
#print(X_train_scaled.describe())
X_train_scaled.shape

(2619, 3)

In [20]:
# Sequential model
def sequential_model(X):
    model = keras.Sequential()
    # Currently unsure of how to add in the intended 3D structure that LSTM layers typically have (Samples, Time Steps, Features)
    input_layer = InputLayer(input_shape = (X.shape[1],), name = 'layer1')
    model.add(input_layer)
    # Hidden layer
    model.add(Dense(64, activation='tanh'))
    # Output layer
    model.add(Dense(3))
    return model

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                256       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 195       
Total params: 451
Trainable params: 451
Non-trainable params: 0
_________________________________________________________________


In [21]:
model = sequential_model(X_train_scaled)
# To check that the list is empty: print(model.layers)
# print(model.summary())
model.compile(loss = 'mse', metrics = ['mae'], optimizer = Adam(learning_rate = 0.01))

In [22]:
model.fit(X_train, y_train, epochs=25, batch_size = 3, verbose=1)
val_mse, val_mae = model.evaluate(X_train_scaled, y_train, verbose = 0)

Epoch 1/25
873/873 [==============================] - 1s 438us/step - loss: 206.1062 - mae: 7.0342
Epoch 2/25
873/873 [==============================] - 0s 440us/step - loss: 106.3249 - mae: 4.9732
Epoch 3/25
873/873 [==============================] - 0s 447us/step - loss: 85.0807 - mae: 4.6701
Epoch 4/25
873/873 [==============================] - 0s 455us/step - loss: 122.1719 - mae: 5.4715
Epoch 5/25
873/873 [==============================] - 0s 441us/step - loss: 87.4621 - mae: 4.8538
Epoch 6/25
873/873 [==============================] - 0s 444us/step - loss: 135.0415 - mae: 6.9039
Epoch 7/25
873/873 [==============================] - 0s 464us/step - loss: 96.1152 - mae: 5.3188
Epoch 8/25
873/873 [==============================] - 0s 449us/step - loss: 67.4516 - mae: 4.3020
Epoch 9/25
873/873 [==============================] - 0s 450us/step - loss: 107.2120 - mae: 4.8847
Epoch 10/25
873/873 [==============================] - 0s 447us/step - loss: 91.9297 - mae: 5.0484
Epoch 11/25
87

In [26]:
test_predict = model.predict_classes(X_test_scaled, batch_size=32, verbose=1)

21/21 [==============================] - 0s 450us/step


C:\ProgramData\Anaconda3\envs\tailoredenv\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [27]:
# Neurons for the layer
layer = layers.Dense(2618)
inputz = tf.ones((2618, 3))
output = layer(inputz)
print(layer.weights)

[<tf.Variable 'dense_6/kernel:0' shape=(3, 2618) dtype=float32, numpy=
array([[ 0.04584765, -0.0211649 , -0.03024502, ..., -0.01228375,
        -0.01559138, -0.00626448],
       [ 0.04046123, -0.00342837,  0.02446431, ..., -0.04512727,
        -0.0289013 ,  0.03293937],
       [ 0.00104909, -0.03528251,  0.01517603, ...,  0.01383283,
         0.02787562,  0.00581347]], dtype=float32)>, <tf.Variable 'dense_6/bias:0' shape=(2618,) dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>]
